# 3. Chunking y Embeddings con LangChain y GitHub Models

## Objetivos de Aprendizaje
- Entender la necesidad de dividir documentos largos en fragmentos (chunking).
- Aprender a usar los divisores de texto de LangChain (`RecursiveCharacterTextSplitter`).
- Comprender qué son los embeddings y por qué son fundamentales para la búsqueda semántica.
- Generar embeddings para los chunks de texto utilizando los modelos de GitHub a través de la API de OpenAI.

## De la Búsqueda por Palabras Clave a la Búsqueda Semántica

En el notebook anterior, construimos un RAG básico que funcionaba con palabras clave. Sus limitaciones eran claras: no entendía el significado, los sinónimos o el contexto. Para superar esto, necesitamos dos procesos clave:

1.  **Chunking**: Si un documento es muy largo, su embedding representará una idea demasiado general. Al dividirlo en chunks más pequeños y cohesivos, cada chunk representa una idea más específica. Esto hace que la búsqueda de similitud sea mucho más precisa.
2.  **Embeddings**: Son la pieza central de la búsqueda semántica. Convierten el texto (nuestros chunks) en vectores numéricos en un espacio multidimensional. Los textos con significados similares tendrán vectores cercanos en este espacio, lo que nos permite encontrar información relevante aunque no se usen las mismas palabras.

In [2]:
import os
import string
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.display import display, Markdown

# --- Configuración del Cliente de OpenAI ---
def initialize_client():
    # Asegúrate de tener las variables de entorno GITHUB_BASE_URL y GITHUB_TOKEN configuradas
    client = OpenAI(
        base_url=os.getenv("GITHUB_BASE_URL", "https://models.inference.ai.azure.com"),
        api_key=os.getenv("GITHUB_TOKEN")
    )
    return client

client = initialize_client()
print("✓ Cliente de OpenAI (compatible con GitHub Models) inicializado.")

✓ Cliente de OpenAI (compatible con GitHub Models) inicializado.


## 1. El Documento a Procesar

Comenzamos con un documento de texto más largo que los que usamos en el notebook anterior. Este texto sobre la historia de la inteligencia artificial es ideal para demostrar la necesidad del chunking.

In [3]:
long_text = (
    "La historia de la inteligencia artificial (IA) es una narrativa fascinante de ambición, innovación y perseverancia. "
    "Sus raíces se remontan a la década de 1950, cuando pioneros como Alan Turing plantearon la pregunta de si las máquinas podían pensar. "
    "El término 'inteligencia artificial' fue acuñado por John McCarthy en 1956 en la famosa Conferencia de Dartmouth, considerada el nacimiento oficial de la IA como campo de estudio. "
    "Los primeros años estuvieron marcados por un gran optimismo, con la creación de programas como el Logic Theorist y el General Problem Solver, que podían resolver problemas de lógica y teoremas matemáticos. "
    "Sin embargo, las limitaciones computacionales y la complejidad de los problemas del mundo real llevaron al primer 'invierno de la IA' en la década de 1970, un período de reducción de fondos y escepticismo. "
    "El resurgimiento llegó en la década de 1980 con el auge de los sistemas expertos, programas que encapsulaban el conocimiento de un experto humano en un dominio específico, como el diagnóstico médico (por ejemplo, MYCIN). "
    "Estos sistemas demostraron el valor comercial de la IA, pero su fragilidad y el alto costo de mantenimiento condujeron a un segundo invierno a finales de los 80 y principios de los 90. "
    "La revolución moderna de la IA comenzó a gestarse a finales de los 90 y principios de los 2000, impulsada por tres factores clave: la disponibilidad de grandes volúmenes de datos (Big Data), el desarrollo de hardware más potente (especialmente las GPU) y los avances en algoritmos de aprendizaje automático, en particular las redes neuronales profundas (deep learning). "
    "Hitos como la victoria de Deep Blue de IBM sobre el campeón de ajedrez Garry Kasparov en 1997 y, más tarde, el triunfo de AlphaGo de DeepMind en el juego de Go en 2016, demostraron el poder del aprendizaje por refuerzo y el deep learning. "
    "Hoy, vivimos en la era de los modelos de lenguaje grande (LLM) como GPT y Claude, y los modelos de difusión para la generación de imágenes, que han llevado la IA a la corriente principal, transformando industrias y planteando nuevas preguntas sobre el futuro de la tecnología y la humanidad."
)

display(Markdown("### 📜 Documento Original"))
print(long_text)
print(f"Longitud del texto: {len(long_text)} caracteres.")

### 📜 Documento Original

La historia de la inteligencia artificial (IA) es una narrativa fascinante de ambición, innovación y perseverancia. Sus raíces se remontan a la década de 1950, cuando pioneros como Alan Turing plantearon la pregunta de si las máquinas podían pensar. El término 'inteligencia artificial' fue acuñado por John McCarthy en 1956 en la famosa Conferencia de Dartmouth, considerada el nacimiento oficial de la IA como campo de estudio. Los primeros años estuvieron marcados por un gran optimismo, con la creación de programas como el Logic Theorist y el General Problem Solver, que podían resolver problemas de lógica y teoremas matemáticos. Sin embargo, las limitaciones computacionales y la complejidad de los problemas del mundo real llevaron al primer 'invierno de la IA' en la década de 1970, un período de reducción de fondos y escepticismo. El resurgimiento llegó en la década de 1980 con el auge de los sistemas expertos, programas que encapsulaban el conocimiento de un experto humano en un domini

## 2. Chunking con `RecursiveCharacterTextSplitter`

LangChain ofrece varias herramientas para dividir texto. `RecursiveCharacterTextSplitter` es una de las más recomendadas porque intenta dividir el texto basándose en una jerarquía de separadores (como saltos de línea dobles, saltos de línea simples, espacios, etc.) para mantener los fragmentos lo más coherentes posible.

-   `chunk_size`: Define el tamaño máximo de cada chunk (en caracteres).
-   `chunk_overlap`: Define cuántos caracteres se superponen entre chunks consecutivos. Esto es crucial para no perder el contexto que podría existir justo en el límite entre dos chunks.

In [4]:
# 1. Inicializar el divisor de texto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,  # Tamaño del chunk en caracteres
    chunk_overlap=50,  # Superposición para mantener contexto
    length_function=len
)

# 2. Dividir el documento en chunks
chunks = text_splitter.split_text(long_text)

display(Markdown(f"### 🧩 Documento Dividido en {len(chunks)} Chunks"))

# 3. Mostrar los chunks resultantes
for i, chunk in enumerate(chunks):
    print(f"--- CHUNK {i+1} (Longitud: {len(chunk)}) ---")
    print(chunk)
    print()

### 🧩 Documento Dividido en 7 Chunks

--- CHUNK 1 (Longitud: 349) ---
La historia de la inteligencia artificial (IA) es una narrativa fascinante de ambición, innovación y perseverancia. Sus raíces se remontan a la década de 1950, cuando pioneros como Alan Turing plantearon la pregunta de si las máquinas podían pensar. El término 'inteligencia artificial' fue acuñado por John McCarthy en 1956 en la famosa Conferencia

--- CHUNK 2 (Longitud: 349) ---
John McCarthy en 1956 en la famosa Conferencia de Dartmouth, considerada el nacimiento oficial de la IA como campo de estudio. Los primeros años estuvieron marcados por un gran optimismo, con la creación de programas como el Logic Theorist y el General Problem Solver, que podían resolver problemas de lógica y teoremas matemáticos. Sin embargo, las

--- CHUNK 3 (Longitud: 348) ---
lógica y teoremas matemáticos. Sin embargo, las limitaciones computacionales y la complejidad de los problemas del mundo real llevaron al primer 'invierno de la IA' en la década de 1970, un período de r

## 3. Generación de Embeddings para cada Chunk

Ahora que tenemos nuestros chunks, el siguiente paso es convertirlos en vectores numéricos. Usaremos el modelo `text-embedding-3-small`, que es eficiente y potente.

El resultado será una lista de vectores, donde cada vector corresponde a un chunk de nuestro documento.

In [6]:
def get_embeddings(client, texts):
    """Función para obtener embeddings de una lista de textos."""
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    return [item.embedding for item in response.data]

# Generar embeddings para todos los chunks
try:
    chunk_embeddings = get_embeddings(client, chunks)
    print(f"✓ Se generaron {len(chunk_embeddings)} embeddings.")
    print(f"Dimensión de cada embedding: {len(chunk_embeddings[0])}")
except Exception as e:
    print(f"❌ Error al generar embeddings: {e}")

# Mostrar un ejemplo de un chunk y su embedding correspondiente
if 'chunk_embeddings' in locals() and chunk_embeddings:
    display(Markdown("### 🔍 Ejemplo: Chunk y su Embedding"))
    
    example_index = 0
    example_chunk = chunks[example_index]
    example_embedding = chunk_embeddings[example_index]
    
    print(f"**Texto del Chunk {example_index + 1}:**{example_chunk}")
    print(f"**Embedding (primeros 10 de {len(example_embedding)} dimensiones):**{example_embedding[:10]}...")

✓ Se generaron 7 embeddings.
Dimensión de cada embedding: 1536


### 🔍 Ejemplo: Chunk y su Embedding

**Texto del Chunk 1:**La historia de la inteligencia artificial (IA) es una narrativa fascinante de ambición, innovación y perseverancia. Sus raíces se remontan a la década de 1950, cuando pioneros como Alan Turing plantearon la pregunta de si las máquinas podían pensar. El término 'inteligencia artificial' fue acuñado por John McCarthy en 1956 en la famosa Conferencia
**Embedding (primeros 10 de 1536 dimensiones):**[0.027127915993332863, 0.009841188788414001, 0.00236149481497705, -0.007694374769926071, 0.016413364559412003, -0.01881389319896698, 0.017398947849869728, 0.05039156973361969, -0.010656001977622509, -0.006372132804244757]...


## 4. Conclusiones y Próximos Pasos

¡Felicidades! Has completado los dos pasos fundamentales para pasar de un RAG básico a uno semántico:

1.  **Has dividido un documento largo en chunks manejables y coherentes.**
2.  **Has convertido cada chunk de texto en un vector numérico (embedding) que captura su significado.**

Con esta lista de chunks y su correspondiente lista de embeddings, ahora tienes una **base de conocimiento vectorial**. 

**En el próximo notebook (`3-vector-rag.ipynb`), aprenderás a:**
-   Tomar una consulta del usuario y generar también su embedding.
-   Usar la **similitud coseno** para comparar el vector de la consulta con todos los vectores de los chunks.
-   Seleccionar los chunks más similares (los que tienen la puntuación de similitud más alta) para usarlos como contexto en la generación de la respuesta.